In [26]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import StratifiedKFold

# Load pheno

In [27]:
pheno_p = '/Users/harveyaa/Documents/masters/data/pheno_26-01-22.csv'
pheno = pd.read_csv(pheno_p, index_col=0)

/var/folders/5x/9dwn0lbx633gccy22ts5bskh0000gn/T/ipykernel_89642/1011432404.py:2: DtypeWarning: Columns (7,8,12,13,14,19,20,24,27,31,42,43,44,49,51,59,60,62,64,65,68,101,121,163) have mixed types. Specify dtype option on import or set low_memory=False.
  pheno = pd.read_csv(pheno_p, index_col=0)


# Filter sites w/ enough controls

In [28]:
sites = pheno['SITE'].unique()

sites = [s for s in sites if pheno[((pheno['non_carriers'] == 1)|(pheno['CON_IPC'] == 1)) & (pheno['SITE'] == s)].shape[0] >= 30]

In [29]:
sites

['UCLA_DS1',
 'UKBB11026',
 'UKBB11025',
 'UKBB11027',
 'HSJ',
 'UCLA_CB',
 'Svip1',
 'Svip2',
 'USM',
 'NYU',
 'SZ3',
 'SZ1',
 'SZ2',
 'SZ6',
 'ADHD5',
 'ADHD6',
 'ADHD3',
 'ADHD1']

# Get CV splits

In [30]:
for s in sites:
    if s[:4] != 'UKBB':
        df = pheno[((pheno['non_carriers'] == 1)|(pheno['CON_IPC'] == 1)) & (pheno['SITE'] == s)]
        skf = StratifiedKFold(n_splits=5,shuffle=True)
        res = skf.split(df.index,df['SEX'])
        cols = []

        for _,r in res:
            col = np.zeros(len(df.index))
            col[r] = 1
            cols.append(col)
        splits = pd.DataFrame(cols,columns=df.index, index=['fold_0','fold_1','fold_2','fold_3','fold_4']).transpose()
        splits.to_csv(f'{s}.csv')

/Users/harveyaa/miniconda3/envs/fmri/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/Users/harveyaa/miniconda3/envs/fmri/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


# UKBB

In [31]:
import random

for s in ['UKBB11025','UKBB11026','UKBB11027']:
    for k in [50,100,200,500,1000,1500,2000,2500,3000,3500,4000,4500]:
        df = pheno[((pheno['non_carriers'] == 1)|(pheno['CON_IPC'] == 1)) & (pheno['SITE'] == s)]
        idx = random.sample(df.index.to_list(),k)

        skf = StratifiedKFold(n_splits=5,shuffle=True)
        res = skf.split(idx,df[df.index.isin(idx)]['SEX'])
        cols = []
        for _,r in res:
            col = np.zeros(len(idx))
            col[r] = 1
            cols.append(col)
        splits = pd.DataFrame(cols,columns=idx, index=['fold_0','fold_1','fold_2','fold_3','fold_4']).transpose()
        splits.to_csv(f'{s}_{k}.csv')